In [1]:
!pip install datasets evaluate transformers[sentencepiece] python-docx

In [2]:
from transformers import BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification, BertModel
from transformers import BertConfig
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Dataset
from transformers import TextDataset

In [3]:
import json
# Read the JSON file
path = "/content/drive/MyDrive/model/B_data.json"
with open(path, 'r') as f:
    dataset = json.load(f)

In [4]:
path = "/content/drive/MyDrive/model/training_set.json"
with open(path, 'r') as f:
    test_dataset = json.load(f)

In [5]:
text, intent, ner = [], [], []
for i in dataset:
    text.append(i['text'])
    intent.append(i['intent'])
    ner.append(i['entities'].split())

In [6]:
test_text, test_intent, test_ner = [], [], []
for i in test_dataset:
    test_text.append(i['text'])
    test_intent.append(i['intent'])
    test_ner.append(i['entities'].split())

In [7]:
unique_intents = set(intent)
num_intent_labels = len(unique_intents)

unique_intents, num_intent_labels

({"'Schedule Appointment'",
  "'Schedule Meeting'",
  "'Set Alarm'",
  "'Set Reminder'",
  "'Set Timer'"},
 5)

In [8]:
one_dimensional_ner = [tag for subset in ner for tag in subset ]
unique_ner = set(one_dimensional_ner)
num_ner_labels = len(unique_ner)
unique_ner, num_ner_labels

({'B-DATE',
  'B-DUR',
  'B-TASK',
  'B-TIME',
  'I-DATE',
  'I-DUR',
  'I-TASK',
  'I-TIME',
  'O'},
 9)

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
ner_model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_ner_labels)
intent_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intent_labels)
bert_backbone = BertModel.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You shoul

In [10]:
labels_to_ids_ner = {
    'O': 0,
    'B-DATE': 1,
    'I-DATE': 2,
    'B-TIME': 3,
    'I-TIME': 4,
    'B-TASK': 5,
    'I-TASK': 6,
    'B-DUR': 7,
    'I-DUR': 8
    }

ids_to_labels_ner = {v: k for k, v in labels_to_ids_ner.items()}
ids_to_labels_ner

{0: 'O',
 1: 'B-DATE',
 2: 'I-DATE',
 3: 'B-TIME',
 4: 'I-TIME',
 5: 'B-TASK',
 6: 'I-TASK',
 7: 'B-DUR',
 8: 'I-DUR'}

In [11]:
labels_to_ids_intent = {
    "'Schedule Appointment'": 0,
    "'Schedule Meeting'": 1,
    "'Set Alarm'": 2,
    "'Set Reminder'": 3,
    "'Set Timer'": 4
}

ids_to_labels_intent = {v: k for k, v in labels_to_ids_intent.items()}
ids_to_labels_intent

{0: "'Schedule Appointment'",
 1: "'Schedule Meeting'",
 2: "'Set Alarm'",
 3: "'Set Reminder'",
 4: "'Set Timer'"}

In [12]:
class dataset(Dataset):
    def __init__(self, text, intent, ner, tokenizer, max_len=128):
        self.len = len(text)
        self.text = text
        self.intent = intent
        self.ner = ner
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: get the sentence, ner label, and intent_label
        sentence = self.text[index].strip()
        intent_label = self.intent[index].strip()
        ner_labels = self.ner[index]

        # step 2: use tokenizer to encode a sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" which highlights where each token starts and ends
        encoding = self.tokenizer(
            sentence,
            return_offsets_mapping=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_len
        )

        # step 3: create ner token labels only for first word pieces of each tokenized word
        tokenized_ner_labels = [labels_to_ids_ner[label] for label in ner_labels]
        # create an empty array of -100 of length max_length
        encoded_ner_labels = np.ones(len(encoding['offset_mapping']), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        prev = -1
        for idx, mapping in enumerate(encoding['offset_mapping']):
            if mapping[0] == mapping[1] == 0:
                continue
            if mapping[0] != prev:
                # overwrite label
                encoded_ner_labels[idx] = tokenized_ner_labels[i]
                prev = mapping[1]
                i += 1
            else:
                prev = mapping[1]

        # create intent token labels
        tokenized_intent_label = labels_to_ids_intent[intent_label]

        # step 4: turn everything into Pytorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['ner_labels'] = torch.as_tensor(encoded_ner_labels)
        item['intent_labels'] = torch.as_tensor(tokenized_intent_label)

        return item

    def __len__(self):
        return self.len

In [13]:
training_set = dataset(text, intent, ner, tokenizer)
test_set = dataset(test_text, test_intent, test_ner, tokenizer)

In [14]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[20]['input_ids']), training_set[20]['ner_labels']):
    print(f"{token} -- {label}")

[CLS] -- -100
schedule -- 0
a -- 0
dentist -- 5
appointment -- 6
for -- 0
april -- 1
5th -- 2
at -- 0
11 -- 3
: -- -100
00 -- -100
in -- 4
the -- 4
morning -- 4
. -- -100
[SEP] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PAD] -- -100
[PA

In [15]:
# The dataset is small, batch_size of 1 would not impact the training time significantly
training_loader = DataLoader(training_set, batch_size=1)
test_loader = DataLoader(test_set, batch_size=1)

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
device

'cpu'

In [18]:
ner_model.to(device)
intent_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [19]:
intent_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [20]:
ner_model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [21]:
bert_backbone

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [22]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [23]:
optimizer = torch.optim.Adam([
    {'params': ner_model.parameters()},
    {'params': intent_model.parameters()}
], lr=1e-5)

# Multi Task Learning Arcitecture

In [30]:
class MultiTaskBertModel(BertModel):

    """
    Multi-task Bert model for Named Entity Recognition (NER) and Intent Classification

    Args:
        config (BertConfig): Bert model configuration.
        num_ner_labels (int): The number of labels for NER task.
        num_intent_labels (int): The number of labels for Intent Classification task.
    """

    def __init__(self, config, num_ner_labels, num_intent_labels):
        super().__init__(config)

        self.num_ner_labels = num_ner_labels
        self.num_intent_labels = num_intent_labels

        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.ner_classifier = torch.nn.Linear(config.hidden_size, self.num_ner_labels)
        self.intent_classifier = torch.nn.Linear(config.hidden_size, self.num_intent_labels)

    def forward(self, input_ids=None, attention_mask=None,
                ner_labels=None, intent_labels=None):

        """
        Perform a forward pass through Multi-task Bert model.

        Args:
            input_ids (torch.Tensor): Input token IDs.
            attention_mask (torch.Tensor): Attention mask for input tokens.
            ner_labels (torch.Tensor): Labels for NER task.
            intent_labels (torch.Tensor): Labels for Intent Classification task.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: NER logits or Intent logits.

        Raises:
            ValueError: If neither ner_labels nor intent_labels were provided.
        """

        if ner_labels is not None:
            outputs = super().forward(input_ids=input_ids,
                                      attention_mask=attention_mask)
            sequence_output = outputs[0]
            sequence_output = self.dropout(sequence_output)
            ner_logits = self.ner_classifier(sequence_output)

            ner_loss_fct = torch.nn.CrossEntropyLoss()
            ner_loss = ner_loss_fct(ner_logits.view(-1, self.num_ner_labels), ner_labels.view(-1))

        if intent_labels is not None:
            outputs = super().forward(input_ids=input_ids,
                                      attention_mask=attention_mask)
            pooled_output = outputs[1]
            pooled_output = self.dropout(pooled_output)
            intent_logits = self.intent_classifier(pooled_output)

            intent_loss_fct = torch.nn.CrossEntropyLoss()
            intent_loss = intent_loss_fct(intent_logits.view(-1, self.num_intent_labels), intent_labels.view(-1))

        if ner_labels is None or intent_labels is None:
            raise ValueError("Either ner_labels or intent_labels were not provided.")

        return ner_loss, ner_logits, intent_loss, intent_logits


In [31]:
model = MultiTaskBertModel(config, num_ner_labels, num_intent_labels)

In [32]:
def train(epoch):
    tr_ner_loss, tr_ner_accuracy = 0, 0
    tr_intent_loss, tr_intent_accuracy = 0, 0
    nb_tr_steps = 0
    tr_ner_preds, tr_ner_labels = [], []
    tr_intent_labels, tr_intent_predictions = [], []
    model.train()


    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        ner_labels = batch['ner_labels'].to(device, dtype=torch.long)
        intent_labels = batch['intent_labels'].to(device, dtype=torch.long)

        ner_loss, ner_logits, intent_loss, intent_logits = model(ids, mask, ner_labels, intent_labels)

        comb_loss = ner_loss + intent_loss

        tr_ner_loss += ner_loss.item()
        tr_intent_loss += intent_loss.item()
        nb_tr_steps += 1

        if idx % 5 == 0:
            loss_ner_step = tr_ner_loss / nb_tr_steps
            loss_intent_step = tr_intent_loss / nb_tr_steps
            print(f"Training NER loss per {idx} training steps: {loss_ner_step}")
            print(f"Training INTENT loss per {idx} training steps: {loss_intent_step}")

        # compute training accuracy (FOR NER)
        flattened_ner_targets = ner_labels.view(-1) # shape (batch_size * seq_len)
        active_ner_logits = ner_logits.view(-1, num_ner_labels) # shape (batch_size*seq_len, num_labels)
        flattened_ner_predictions = torch.argmax(active_ner_logits, axis=1) # shape (batch_size * seq_len)

        # compute accuracy only at active labels
        active_ner_accuracy = ner_labels.view(-1) != -100 # shape (batch_size, seq_len)
        ac_ner_labels = torch.masked_select(flattened_ner_targets, active_ner_accuracy)
        ner_predictions = torch.masked_select(flattened_ner_predictions, active_ner_accuracy)

        tr_ner_labels.extend(ac_ner_labels)
        tr_ner_preds.extend(ner_predictions)

        # compute accuracy for intent_model
        # I CAN MAKE THE CALCULATION MUCH EASIER
        # FIGURE IT OUT
        flattened_intent_targets = intent_labels.view(-1)
        active_intent_logits = intent_logits.view(-1, num_intent_labels)
        flattened_intent_predictions = torch.argmax(active_intent_logits, axis=1)

        sample_intent_accuracy = intent_labels.view(-1)
        active_intent_accuracy = torch.ones_like(sample_intent_accuracy, dtype=torch.bool)

        ac_intent_labels = torch.masked_select(flattened_intent_targets, active_intent_accuracy)
        intent_predictions = torch.masked_select(flattened_intent_predictions, active_intent_accuracy)

        tr_intent_labels.extend(ac_intent_labels)
        tr_intent_predictions.extend(intent_predictions)

        tmp_tr_ner_accuracy = accuracy_score(ac_ner_labels.cpu().numpy(), ner_predictions.cpu().numpy())
        tmp_tr_intent_accuracy = accuracy_score(ac_intent_labels.cpu().numpy(), intent_predictions.cpu().numpy())


        tr_ner_accuracy += tmp_tr_ner_accuracy
        tr_intent_accuracy += tmp_tr_intent_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters = model.parameters(), max_norm=10
        )

        # backward pass
        optimizer.zero_grad()
        comb_loss.backward()
        optimizer.step()

    epoch_loss = (tr_ner_loss + tr_intent_loss) / nb_tr_steps
    tr_ner_accuracy = tr_ner_accuracy / nb_tr_steps
    tr_intent_accuracy = tr_intent_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training NER accuracy epoch: {tr_ner_accuracy}")
    print(f"Training INTENT accuracy epoch: {tr_intent_accuracy}")

In [33]:
for epoch in range(3):
    print(f"Training epoch: {epoch+1}")
    print("-----------------------------")
    train(epoch)

Training epoch: 1
-----------------------------
Training NER loss per 0 training steps: 2.1673552989959717
Training INTENT loss per 0 training steps: 1.6900688409805298
Training NER loss per 5 training steps: 2.1874396602312722
Training INTENT loss per 5 training steps: 1.6025922497113545
Training NER loss per 10 training steps: 2.258472616022283
Training INTENT loss per 10 training steps: 1.559900630604137
Training NER loss per 15 training steps: 2.317053332924843
Training INTENT loss per 15 training steps: 1.6326571330428123
Training NER loss per 20 training steps: 2.2916664509546187
Training INTENT loss per 20 training steps: 1.6293905008406866
Training NER loss per 25 training steps: 2.3174009048021755
Training INTENT loss per 25 training steps: 1.6521743215047395
Training NER loss per 30 training steps: 2.301511802980977
Training INTENT loss per 30 training steps: 1.6308588866264588
Training loss epoch: 3.9352242367608206
Training NER accuracy epoch: 0.09088625660054232
Training I